In [1]:
%load_ext watermark
%watermark -a "Ruiyu Hu" -d -v -m

Ruiyu Hu 2019-03-17 

CPython 3.6.7
IPython 7.0.1

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 58 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [2]:
# read the pdf file
import PyPDF2 

# tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob

# create inverse index

import math
import glob
import json
import operator
from collections import Counter,defaultdict



import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

**Create Tokenize**

In [9]:
#%%file tokenize.py

from textblob import TextBlob

def get_file_names():
    files = []
    for file in glob.glob("../data/text/*.txt"):
        files.append(file)
    return files
###########################################################################
def convert(file):
    text = []
    with open(str(file), 'rb') as f:
        for line in f.readlines():
            text.append(line.decode("utf-8", "ignore").strip())
    return text

###########################################################################     
def tokenize(lst):            
    # create a list of token
    
    tokens = [None] * len(lst)
    for i in range(len(lst)):
        tokens[i] = clean_token(lst[i])
    tokens = [t for tok in tokens for t in tok] 
    return tokens

########################################################################### 
def clean_token(text):
    #porter = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    #snowball = nltk.SnowballStemmer('english')
        
    stopset = set(stopwords.words('english'))
    stopset.update(('less','year'))
    
    noun_lst = []

    for word,tag in (TextBlob(text).tags):
        if tag in ("NN", "NNS", "NNP", "NNPS","JJ"):
            word = word.lower()
            word = lemmatizer.lemmatize(word)
            #word = porter.stem(word)
            #word = snowball.stem(word)
            if word not in stopset and word.isalpha() and len(word)>2:
                noun_lst.append(word)
    return noun_lst

**create index**

In [3]:
def get_file_names():
    files = []
    for file in glob.glob("../data/text/*.txt"):
        files.append(file)
    return files

get_file_names()[:2]

['../data/text\\100128282NAKUL KUMAR_original.txt',
 '../data/text\\100276799CV_ajoy.txt']

In [12]:
def get_file_names():
    files = []
    for file in glob.glob("../data/text/*.txt"):
        files.append(file)
    return files

#get_file_names()
def make_idx(tokens, doc_name, idx, length):
    for term in set(tokens):
        idx[term].append([doc_name,tokens.count(term)])
        length[doc_name] = len(set(tokens))

def write(inverted_idx,len_idx):
    inv_idx_file = open("../data/indexes/inverted_idx.json","w")
    json.dump(inverted_idx,inv_idx_file)

    len_idx_file = open("../data/indexes/len_idx.json","w")
    json.dump(len_idx,len_idx_file)
    
def generate_idx():
    resume_files = get_file_names()
    inverted_index = defaultdict(list)
    length_index = defaultdict(list)
    
    for file in resume_files:
        make_idx(tokenize(convert(file)), file, inverted_index, length_index)
        
    write(inverted_index,length_index)
    print ("Indexes generated")
    

In [13]:
generate_idx()

Indexes generated


In [ ]:
'''inv_idx_file = open("../data/indexes/inverted_idx.json","r")
inv_indx = json.load(inv_idx_file)
# create dictionary
doc_freq ={}
for key in sorted(inv_indx.keys()):
    doc_freq[key] = sum(Counter(set(doc_id for doc_id, term in inv_indx[key])).values())

dictionary = pd.DataFrame.from_dict(doc_freq,orient='index', columns=['DocFreq'])
dictionary.head()'''

**create retrieval-The BM25 Weighting Scheme**

In [14]:
from math import log

'''
IR Book: 11.4.3
Fomula: 11.33
'''
'''
typical TREC value (Text Retrieval Conference (TREC).)
f1 = 1.2
k2 varies from 0 to 1000
b = 0.75
'''

k1 = 1.2
b = 0.75
k2 = 300
R = 0 # (set it to 0 since no relevancy info is known)



def BM25(doc_len, avg_doc_len, n_doc_w_term, n_total_doc, freq_term_doc, freq_term_query, rel_doc_w_term):
    
    n = n_doc_w_term
    N = n_total_doc
    f = freq_term_doc
    q = freq_term_query
    r = rel_doc_w_term
    
    p1 = ((k2 + 1) * q) / (k2 + q) #Relevance between term and query
    p2 = ((k1 + 1) * f) / (getK(doc_len, avg_doc_len) + f) #Relevance between term and document
    p3 = log((((r + 0.5)/(R-r+0.5)) / ((n - r + 0.5)/(N - n - R + r + 0.5)))+1) # Term Weight
    return p1 * p2 * p3

def getK(doc_len, avg_doc_len):
    return k1 * ((1 - b) + b * (float(doc_len) / float(avg_doc_len)))
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [23]:
# get average document length
def get_avg_doc_len(len_idx):
    _length = 0
    for doc in len_idx:
        _length += len_idx[doc]
    return float(_length) / float(len(len_idx))

def search(query):
    generate_idx()
    inv_idx_file = open("../data/indexes/inverted_idx.json","r")
    inverted_idx = json.load(inv_idx_file)

    len_idx_file = open("../data/indexes/len_idx.json","r")
    len_idx = json.load(len_idx_file)

    scores = defaultdict(list)
    
    query_tokens = query.split()
    for token in query_tokens:
    #for token in query:
        if token in inverted_idx.keys():
            for entry in inverted_idx[token]:
                bm25_val = BM25(len_idx[entry[0]],get_avg_doc_len(len_idx),len(inverted_idx[token]),len(len_idx),entry[1],1,0)
                scores[entry[0]] = round(bm25_val,4)
    result = sorted(scores.items(),key=operator.itemgetter(1),reverse=True)
    return result

In [24]:
search('electrical engineer')[:10]

Indexes generated


[('../data/text\\105377002jobin resume.txt', 1.6662),
 ('../data/text\\114030789fazle_resume.txt', 1.5096),
 ('../data/text\\105117109GAURAV RESUME.txt', 1.4695),
 ('../data/text\\119391331Arshad cv new 2015.txt', 1.4537),
 ('../data/text\\108877856Resume_-_Iqbal.txt', 1.4284),
 ('../data/text\\116252012oct21myelectricalcv2011.txt', 1.3691),
 ('../data/text\\112216114ehtesham_resume.txt', 1.3211),
 ('../data/text\\100276799CV_ajoy.txt', 1.2701),
 ('../data/text\\114891715selvam_mep.txt', 1.2341),
 ('../data/text\\115876417RESUME.txt', 1.2268)]

**w2v approach**

In [56]:
import time
from gensim.models import KeyedVectors
t1 = time.time()
path = 'C:/Users/RayHu/Downloads/google_w2v/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(path, binary=True)
print('-------------------------------------------')
print("Loading word2vec model cost %.3f seconds...\n" % (time.time() - t1))

-------------------------------------------
Loading word2vec model cost 108.422 seconds...



In [147]:
%%file vectorize.py

import time
import numpy as np
from numpy import linalg as LA
from gensim.models import KeyedVectors
import tokenize

t1 = time.time()
#download link: https://github.com/mmihaltz/word2vec-GoogleNews-vectors
path = 'C:/Users/RayHu/Downloads/google_w2v/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(path, binary=True)
print('-------------------------------------------')
print("Loading word2vec model cost %.3f seconds...\n" % (time.time() - t1))

# -------------------------------------------
#Loading word2vec model cost 108.422 seconds...

def vectorize(words):
    '''
    transform the doc and query into vectors
    '''
    word_vec = []
    for word in words:
        try:
            vec = model[word]
            word_vec.append(vec)
        except KeyError:
            # ignore the word if it is not in the w2v vocablary
            pass
    vector = np.mean(word_vec,axis = 0)
    return vector

def cos_sim(vector1, vector2):
    '''
    the fomula to calculte cosine cimilarity 
    '''
    sim = np.dot(vector1, vector2) / (LA.norm(vector1) * LA.norm(vector2))
    
    if np.isnan(np.sum(sim)):
        return 0
    
    return sim

def calc_sim(query, threshold = 0):
    '''
    calculate similarity scores between documents and the query
    '''
    query = clean_token(query)
    file_list = get_file_names()
    documents = {}
    
    for i in range(len(file_list)):
        documents[file_list[i]] = tokenize(convert(file_list[i]))
        
    query_vec = vectorize(query)
    results = {}
    
    for name, doc in documents.items():
        doc_vec = vectorize(doc)
        sim_score = cos_sim(query_vec, doc_vec)
        if sim_score > threshold:
            results[name] = sim_score
            sort_result = sorted(results.items(),key=operator.itemgetter(1),reverse=True)
    return sort_result

Overwriting vectorize.py


In [ ]:
query = 'define business metric and perform insight analysis'
calc_sim(query, 0)[:5]

**create Ranker**

In [138]:
def matching(query):
    results_bm25 = search(query)[:5]
    results_cos = calc_sim(query, 0)[:5]
    print('Ranking by BM25')
    for result in results_bm25:
        print(result)
    print('Ranking by cosine similarity')
    for result in results_cos:
        print(result)
    #return results_bm25, results_cos

In [139]:
matching('define business metric, perform insight analysis')

Indexes generated
Ranking by BM25
('../data/documents\\de002.pdf', 2.5891)
('../data/documents\\ds004.pdf', 2.4813)
('../data/documents\\da005.pdf', 1.5244)
('../data/documents\\ds002.pdf', 1.4832)
('../data/documents\\ds001.pdf', 1.436)
Ranking by cosine similarity
('../data/documents\\ds002.pdf', 0.657538)
('../data/documents\\da004.pdf', 0.6372742)
('../data/documents\\da003.pdf', 0.63445765)
('../data/documents\\de005.pdf', 0.619592)
('../data/documents\\ds001.pdf', 0.6021874)


## Jd side

In [131]:
def clean_jd(jd):
    '''
    method 2: terms sorted by tf idf weights
    '''
    doc = clean_token(jd)
    
    cv=CountVectorizer()
    word_count_vector=cv.fit_transform(doc)
    
    #calculate the weights for each term in each document
    tfidf_transformer=TfidfTransformer()
    tf_idf_vector = tfidf_transformer.fit_transform(word_count_vector)
    #the top 20 terms by average tf-idf weight
    weights = np.asarray(tf_idf_vector.mean(axis=0)).ravel().tolist()
    weights_df = pd.DataFrame({'term': cv.get_feature_names(), 'weight': weights}).sort_values(by='weight', ascending=False).head(10)
    return weights_df.term.tolist()

## Search

In [121]:
dirt = '../data/job_description/ml-jd-adobe.pdf'

In [22]:
# method 2
#print(matching(clean_jd(dirt)))

In [132]:
print ("Enter search query")
jd = input(":: ")
keywords = clean_jd(jd)
results = matching(keywords)
print("\nThe Matching Resumes Are:")
#for result in results:
    #print (result[0])
print(results)

Enter search query


::  Experience with supervised and unsupervised machine learning algorithms, and ensemble methods


Indexes generated


AttributeError: 'list' object has no attribute 'split'